In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Cargar el dataset procesado
df = pd.read_csv('../data/semiprocessed_data.csv')

# Separar variables predictoras y objetivo
X = df.drop("Price", axis=1)
y = df["Price"]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Vamos a empezar con el modelo de Redes Neuronales y resumirlo

In [3]:
# Definir el modelo
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Primera capa oculta con 64 neuronas
    Dense(32, activation='relu'),                                  # Segunda capa oculta con 32 neuronas
    Dense(1)                                                      # Capa de salida para regresión
])

# Resumen del modelo
model.summary()


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,433 (9.50 KB)

 Trainable params: 2,433 (9.50 KB)

 Non-trainable params: 0 (0.00 B)

Ahora vamos a compilar el modelo y luego entrenarlo. 

In [4]:
model.compile(
    optimizer='adam',            
    loss='mean_squared_error',      
    metrics=['mean_absolute_error'] 
)

print("Modelo compilado correctamente.")


Modelo compilado correctamente.


In [5]:
# Entrenar el modelo
history = model.fit(
    X_train,          # Datos de entrenamiento
    y_train,          # Etiquetas de entrenamiento
    validation_split=0.2,  # Usar el 20% de los datos de entrenamiento para validación
    epochs=50,        # Número de épocas
    batch_size=32,    # Tamaño del batch
    verbose=1         # Modo de salida (1 para ver detalles del progreso)
)

print("Entrenamiento completado.")


Epoch 1/50
17043/17043 ━━━━━━━━━━━━━━━━━━━━ 5s 273us/step - loss: 197263104.0000 - mean_absolute_error: 9369.1133 - val_loss: 187589040.0000 - val_mean_absolute_error: 8883.8125
Epoch 2/50
17043/17043 ━━━━━━━━━━━━━━━━━━━━ 5s 266us/step - loss: 176646800.0000 - mean_absolute_error: 8770.7578 - val_loss: 186514176.0000 - val_mean_absolute_error: 8764.0254
Epoch 3/50
17043/17043 ━━━━━━━━━━━━━━━━━━━━ 4s 263us/step - loss: 174422368.0000 - mean_absolute_error: 8738.5635 - val_loss: 186156224.0000 - val_mean_absolute_error: 8845.0723
Epoch 4/50
17043/17043 ━━━━━━━━━━━━━━━━━━━━ 4s 262us/step - loss: 173199968.0000 - mean_absolute_error: 8704.6436 - val_loss: 185386016.0000 - val_mean_absolute_error: 8606.3701
Epoch 5/50
17043/17043 ━━━━━━━━━━━━━━━━━━━━ 4s 262us/step - loss: 173905408.0000 - mean_absolute_error: 8681.9902 - val_loss: 184774848.0000 - val_mean_absolute_error: 8588.5332
Epoch 6/50
17043/17043 ━━━━━━━━━━━━━━━━━━━━ 5s 264us/step - loss: 174181296.0000 - mean_absolute_error: 8679.5

In [6]:
# Evaluar el modelo en el conjunto de prueba
test_loss = model.evaluate(X_test, y_test, verbose=0)
print(f"Pérdida (MSE) en el conjunto de prueba: {test_loss}")

# Realizar predicciones
y_pred_nn = model.predict(X_test)

# Calcular métricas adicionales
from sklearn.metrics import mean_absolute_error, r2_score

mae_nn = mean_absolute_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)

print(f"MAE del modelo de Red Neuronal: {mae_nn}")
print(f"R² del modelo de Red Neuronal: {r2_nn}")


Pérdida (MSE) en el conjunto de prueba: [108402944.0, 6009.14501953125]
5326/5326 ━━━━━━━━━━━━━━━━━━━━ 1s 179us/step
MAE del modelo de Red Neuronal: 6009.14576541289
R² del modelo de Red Neuronal: 0.41039339273693454


Vamos a ajustar la arquitectura agregando capas ocultas

In [7]:
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),  
    Dense(1) 
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mean_absolute_error'])

model.summary()

history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)

loss, mae = model.evaluate(X_test, y_test, verbose=1)
r2 = 1 - (loss / ((y_test - y_test.mean()) ** 2).sum())

print(f"Pérdida (MSE) en el conjunto de prueba: {loss}")
print(f"MAE del modelo de Red Neuronal: {mae}")
print(f"R² del modelo de Red Neuronal: {r2}")

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
17043/17043 ━━━━━━━━━━━━━━━━━━━━ 5s 277us/step - loss: 194009776.0000 - mean_absolute_error: 9375.3506 - val_loss: 187008880.0000 - val_mean_absolute_error: 8705.4443
Epoch 2/50
17043/17043 ━━━━━━━━━━━━━━━━━━━━ 5s 294us/step - loss: 175154240.0000 - mean_absolute_error: 8759.9385 - val_loss: 185180544.0000 - val_mean_absolute_error: 8657.6729
Epoch 3/50
17043/17043 ━━━━━━━━━━━━━━━━━━━━ 5s 283us/step - loss: 173910944.0000 - mean_absolute_error: 8671.2236 - val_loss: 185587216.0000 - val_mean_absolute_error: 8898.2676
Epoch 4/50
17043/17043 ━━━━━━━━━━━━━━━━━━━━ 5s 281us/step - loss: 172248464.0000 - mean_absolute_error: 8630.8340 - val_loss: 182280880.0000 - val_mean_absolute_error: 8757.1895
Epoch 5/50
17043/17043 ━━━━━━━━━━━━━━━━━━━━ 5s 281us/step - loss: 172180992.0000 - mean_absolute_error: 8614.4863 - val_loss: 180603216.0000 - val_mean_absolute_error: 8721.1006
Epoch 6/50
17043/17043 ━━━━━━━━━━━━━━━━━━━━ 5s 279us/step - loss: 168969920.0000 - mean_absolute_error: 8565.1

Ahora muestran resultados buenos pero de pronto mentirosos, entonces vamos a ver si los datos son representativos y consistentes. Ajustaremos la cantidad de epocas y tamaño del batch

In [8]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=30, batch_size=64, verbose=1)

loss, mae = model.evaluate(X_test, y_test, verbose=1)
r2 = 1 - (loss / ((y_test - y_test.mean()) ** 2).sum())

print(f"Pérdida (MSE) en el conjunto de prueba: {loss}")
print(f"MAE del modelo de Red Neuronal: {mae}")
print(f"R² del modelo de Red Neuronal: {r2}")


Epoch 1/30
8522/8522 ━━━━━━━━━━━━━━━━━━━━ 2s 288us/step - loss: 106488176.0000 - mean_absolute_error: 5850.5454 - val_loss: 109440016.0000 - val_mean_absolute_error: 5755.2368
Epoch 2/30
8522/8522 ━━━━━━━━━━━━━━━━━━━━ 3s 294us/step - loss: 104402992.0000 - mean_absolute_error: 5825.4956 - val_loss: 111219000.0000 - val_mean_absolute_error: 5889.1699
Epoch 3/30
8522/8522 ━━━━━━━━━━━━━━━━━━━━ 3s 301us/step - loss: 99533488.0000 - mean_absolute_error: 5802.8521 - val_loss: 114392224.0000 - val_mean_absolute_error: 6124.6855
Epoch 4/30
8522/8522 ━━━━━━━━━━━━━━━━━━━━ 2s 291us/step - loss: 104284184.0000 - mean_absolute_error: 5803.9648 - val_loss: 108080832.0000 - val_mean_absolute_error: 5549.3799
Epoch 5/30
8522/8522 ━━━━━━━━━━━━━━━━━━━━ 3s 294us/step - loss: 100335792.0000 - mean_absolute_error: 5791.4966 - val_loss: 107921728.0000 - val_mean_absolute_error: 5546.5083
Epoch 6/30
8522/8522 ━━━━━━━━━━━━━━━━━━━━ 2s 291us/step - loss: 101896432.0000 - mean_absolute_error: 5745.1792 - val_los

Ahora vamos a cambiar la cantidad de neuronas en las capas ocultas, incrementando las neuronas en las capas ocultas y ajustando la tasa de aprendizaje

In [9]:
model = Sequential([
    Dense(128, activation='relu', input_dim=X_train.shape[1]),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')  
])

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,  
    batch_size=64, 
    verbose=1
)

mse, mae = model.evaluate(X_test, y_test, verbose=0)
r2 = 1 - (mse / ((y_test - y_test.mean())**2).sum())

print("MSE del modelo de Red Neuronal:", mse)
print("MAE del modelo de Red Neuronal:", mae)
print("R² del modelo de Red Neuronal:", r2)


Epoch 1/30


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10652/10652 ━━━━━━━━━━━━━━━━━━━━ 5s 382us/step - loss: 189389312.0000 - mae: 9163.3330 - val_loss: 177810480.0000 - val_mae: 8951.3184
Epoch 2/30
10652/10652 ━━━━━━━━━━━━━━━━━━━━ 4s 394us/step - loss: 177476320.0000 - mae: 8679.2451 - val_loss: 173190992.0000 - val_mae: 8878.3574
Epoch 3/30
10652/10652 ━━━━━━━━━━━━━━━━━━━━ 5s 443us/step - loss: 172520832.0000 - mae: 8540.0879 - val_loss: 170441088.0000 - val_mae: 8461.8770
Epoch 4/30
10652/10652 ━━━━━━━━━━━━━━━━━━━━ 4s 371us/step - loss: 164540064.0000 - mae: 8329.6738 - val_loss: 156313120.0000 - val_mae: 8323.2812
Epoch 5/30
10652/10652 ━━━━━━━━━━━━━━━━━━━━ 4s 370us/step - loss: 153758304.0000 - mae: 7858.3223 - val_loss: 148810320.0000 - val_mae: 7852.1626
Epoch 6/30
10652/10652 ━━━━━━━━━━━━━━━━━━━━ 6s 579us/step - loss: 146086624.0000 - mae: 7493.5117 - val_loss: 131229432.0000 - val_mae: 6996.9507
Epoch 7/30
10652/10652 ━━━━━━━━━━━━━━━━━━━━ 5s 426us/step - loss: 136889680.0000 - mae: 7102.7095 - val_loss: 121384256.0000 - val_mae:

In [10]:
model.save('red_neuronal_model.h5')
print("Modelo guardado exitosamente como 'red_neuronal_model.h5'")

Modelo guardado exitosamente como 'red_neuronal_model.h5'
